# Setting working directory

## Load the Google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Change the workding dir to: 

`'/content/drive/My Drive/Colab Notebooks/MachineLearningPractice/FinanceRiskControl'`

In [ ]:
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/MachineLearningPractice/FinanceRiskControl')
!ls

2-DataEDA.ipynb					  4-Tweaking.ipynb
3.0-FeatureEngineering-original.ipynb		  5.1-Ensemble-Stacking.ipynb
3.1-FeatureEngineering-LagrangeInterpolate.ipynb  5.1-XGB-HPTweaking.ipynb
3.2-FeatureEngineering-OtherChanges.ipynb	  6-Ensemble.ipynb
3-FeatureEngineering.ipynb			  originalDataset
4.1-CAT-smallMemSize.ipynb			  preprocessedData
4.1-LGB-smallMemSize.ipynb			  submissionResults
4.1-XGB-smallMemSize.ipynb


## Go to this place for original dataset: 

`'/content/drive/My Drive/Colab Notebooks/MachineLearningPractice/FinanceRiskControl/originalDataset'`

# Importing libraries

In [ ]:
# !pip install catboost

     |████████████████████████████████| 66.2MB 50kB/s 


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb
import lightgbm as lgb
# from catboost import CatBoostRegressor
import warnings
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, log_loss
import tqdm
warnings.filterwarnings('ignore')

# Starting feature engineering

## Load dataset

### Original dataset

In [ ]:
data_train = pd.read_csv('originalDataset/train.csv')
data_test_a = pd.read_csv('originalDataset/testA.csv')

### Load some preprocessed dataset

In [ ]:
data_train = pd.read_csv("preprocessedData/lagrangeInterpolated_train.csv")
data_test_a = pd.read_csv("preprocessedData/lagrangeInterpolated_test.csv")

In [ ]:
numerical_fea = list(data_train.select_dtypes(exclude=['object']).columns)
category_fea = list(filter(lambda x: x not in numerical_fea,list(data_train.columns)))
label = 'isDefault'
numerical_fea.remove(label)

## Change `employmentLength` type into regular int

In [ ]:
def employmentLength_to_int(s):
    if pd.isnull(s):
        return s
    else:
        return np.int8(s.split()[0]) ## mind this. use the int type in np, not general python int. 
    
for data in [data_train, data_test_a]:
    data['employmentLength'].replace(to_replace='10+ years', value='10 years', inplace=True)
    data['employmentLength'].replace('< 1 year', '0 years', inplace=True)
    data['employmentLength'] = data['employmentLength'].apply(employmentLength_to_int)

## Fill the null. 

**Mind this**: Some other filling schemes can be used. 

In [ ]:
data_train.isnull().sum()

id                        0
loanAmnt                  0
term                      0
interestRate              0
installment               0
grade                     0
subGrade                  0
employmentTitle           1
employmentLength      46799
homeOwnership             0
annualIncome              0
verificationStatus        0
issueDate                 0
isDefault                 0
purpose                   0
postCode                  1
regionCode                0
dti                     239
delinquency_2years        0
ficoRangeLow              0
ficoRangeHigh             0
openAcc                   0
pubRec                    0
pubRecBankruptcies      405
revolBal                  0
revolUtil               531
totalAcc                  0
initialListStatus         0
applicationType           0
earliesCreditLine         0
title                     1
policyCode                0
n0                    40270
n1                    40270
n2                    40270
n3                  

In [ ]:
data_train["n14"].head()

0    2.0
1    NaN
2    4.0
3    1.0
4    4.0
Name: n14, dtype: float64

Change the infinite number into NaN. 

In [ ]:
# data_train.replace([np.inf, -np.inf], np.nan, inplace=True)
# data_test_a.replace([np.inf, -np.inf], np.nan, inplace=True)

### Lagrange interpolation

In [ ]:
# data_train_1 = data_train.head(100)
# data_test_a_1 = data_test_a.head(100)

In [ ]:
# 创建函数，做插值，以空值前后5个数据（共10个数据）为例做插值  
from scipy.interpolate import lagrange  

## https://www.programmersought.com/article/37145216331/
def fillNanWithLagr(col,nv=-1,k=3):
    # col "fill column vector", nv "empty value, default -1", k "Lagrangian interval, default 3"
    # Get the null position
    if nv is np.nan:
        tar = col[col.isnull()].index.tolist()
    else:
        tar = col[col==nv].index.tolist()

    for idx in tqdm.tqdm(tar, position=0, leave=True):
      ## you can also use tqdm.tqdm_notebook(). The graphic info is more beautiful. 
    # for idx in tqdm.tqdm(tar):
                # Get Lagrange interval
                # The empty value position is removed here, because the filled empty value cannot be used to calculate other empty values
        # print(idx, end="")
        rel = col.iloc[
            list(
                set(list(range(idx-k,idx))+list(range(idx+1,idx+k+1))).difference(set(tar))
            )
        ]
                # Keep a reasonable range
        rel = rel[rel>=0][rel<len(col)]
                # Fill in Lagrangian mean
        # lagrange(arg1,arg2)(arg3)
                # arg1 "Iterable object index", arg2 "Iterable object conversion table", arg3 "Fill position"
        col.iloc[idx] = lagrange(rel.index,list(rel))(idx)
    return col

In [ ]:
data_train["employmentLength"].head()

0     2.0
1     5.0
2     8.0
3    10.0
4     NaN
Name: employmentLength, dtype: float64

In [ ]:
for i, data in enumerate([data_train, data_test_a]):
    print("in the {} dataset:".format(i + 1))
    for fea in ["employmentLength"]:
        if fea == "id":
            continue
        print(fea)
        data[fea] = fillNanWithLagr(data[fea], np.nan, 5)

  0%|          | 1/46799 [00:00<1:34:31,  8.25it/s]

in the 1 dataset:
employmentLength


  0%|          | 13/11742 [00:00<01:30, 129.71it/s]

in the 2 dataset:
employmentLength


100%|██████████| 11742/11742 [01:06<00:00, 176.58it/s]


In [ ]:
# data_train_cp = data_train.copy()
# data_test_a_cp = data_test_a.copy()

for i, data in enumerate([data_train, data_test_a]):
    print("in the {} dataset:".format(i + 1))
    for fea in numerical_fea:
        if fea == "id":
            continue
        print(fea)
        data[fea] = fillNanWithLagr(data[fea], np.nan, 5)

# interpolate()

0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 1/1 [00:00<00:00, 48.66it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 1/1 [00:00<00:00, 89.36it/s]
0it [00:00, ?it/s]
  0%|          | 0/239 [00:00<?, ?it/s]

in the 1 dataset:
loanAmnt
term
interestRate
installment
employmentTitle
homeOwnership
annualIncome
verificationStatus
purpose
postCode
regionCode
dti


100%|██████████| 239/239 [00:01<00:00, 126.36it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
  3%|▎         | 11/405 [00:00<00:03, 106.39it/s]

delinquency_2years
ficoRangeLow
ficoRangeHigh
openAcc
pubRec
pubRecBankruptcies


100%|██████████| 405/405 [00:03<00:00, 123.25it/s]
0it [00:00, ?it/s]
  2%|▏         | 12/531 [00:00<00:04, 113.27it/s]

...

  0%|          | 12/10111 [00:00<01:25, 117.76it/s]

n13


  0%|          | 25/10111 [00:00<01:23, 120.09it/s]

n14


100%|██████████| 10111/10111 [01:15<00:00, 134.14it/s]


In [ ]:
data_train.isnull().sum()

id                    0
loanAmnt              0
term                  0
interestRate          0
installment           0
grade                 0
subGrade              0
employmentTitle       0
employmentLength      0
homeOwnership         0
annualIncome          0
verificationStatus    0
issueDate             0
isDefault             0
purpose               0
postCode              0
regionCode            0
dti                   0
delinquency_2years    0
ficoRangeLow          0
ficoRangeHigh         0
openAcc               0
pubRec                0
pubRecBankruptcies    0
revolBal              0
revolUtil             0
totalAcc              0
initialListStatus     0
applicationType       0
earliesCreditLine     0
title                 0
policyCode            0
n0                    0
n1                    0
n2                    0
n3                    0
n4                    0
n5                    0
n6                    0
n7                    0
n8                    0
n9              

In [ ]:
data_test_a.isnull().sum()

id                    0
loanAmnt              0
term                  0
interestRate          0
installment           0
grade                 0
subGrade              0
employmentTitle       0
employmentLength      0
homeOwnership         0
annualIncome          0
verificationStatus    0
issueDate             0
purpose               0
postCode              0
regionCode            0
dti                   0
delinquency_2years    0
ficoRangeLow          0
ficoRangeHigh         0
openAcc               0
pubRec                0
pubRecBankruptcies    0
revolBal              0
revolUtil             0
totalAcc              0
initialListStatus     0
applicationType       0
earliesCreditLine     0
title                 0
policyCode            0
n0                    0
n1                    0
n2                    0
n3                    0
n4                    0
n5                    0
n6                    0
n7                    0
n8                    0
n9                    0
n10             

In [ ]:
data_train.to_csv("preprocessedData/lagrangeInterpolated_train-1.csv", index=False)
data_test_a.to_csv("preprocessedData/lagrangeInterpolated_test-1.csv", index=False)

### Median interpolation

In [ ]:
#按照平均数填充数值型特征
data_train[numerical_fea] = data_train[numerical_fea].fillna(data_train[numerical_fea].median())
data_test_a[numerical_fea] = data_test_a[numerical_fea].fillna(data_train[numerical_fea].median())
#按照众数填充类别型特征
data_train[category_fea] = data_train[category_fea].fillna(data_train[category_fea].mode())
data_test_a[category_fea] = data_test_a[category_fea].fillna(data_train[category_fea].mode())

In [ ]:
# data_train = data_train.fillna(axis = 0, method = "ffill")

## Process the dates

From the `original dates` to `number of days from a certain starting date`.

In [ ]:
#转化成时间格式
startdate = datetime.datetime.strptime('2007-06-01', '%Y-%m-%d')
for data in [data_train, data_test_a]:
    data['issueDate'] = pd.to_datetime(data['issueDate'],format='%Y-%m-%d')
    #构造时间特征
    data['issueDateDT'] = data['issueDate'].apply(lambda x: x-startdate).dt.days

## Change the `object` type into a numerical type 

In [ ]:
# data_train['employmentLength'].value_counts(dropna=False).sort_index()

In [ ]:
def employmentLength_to_int(s):
    if pd.isnull(s):
        return s
    else:
        return np.int8(s.split()[0]) ## mind this. use the int type in np, not general python int. 
    
for data in [data_train, data_test_a]:
    data['employmentLength'].replace(to_replace='10+ years', value='10 years', inplace=True)
    data['employmentLength'].replace('< 1 year', '0 years', inplace=True)
    data['employmentLength'] = data['employmentLength'].apply(employmentLength_to_int)

**Mind this**: the `earliesCreditLine` was MMM-YYYY, but after the following preprocess, this column only has YYYY. 

So in the future, we may find other ways to preprocess this column. 

In [ ]:
for data in [data_train, data_test_a]:
    data['earliesCreditLine'] = data['earliesCreditLine'].apply(lambda x: int(x[-4:]))

In [ ]:
# data_train['earliesCreditLine'].sample(5)

## Use label encoding to change other object type columns. 

In [ ]:
for data in [data_train, data_test_a]:
    data['grade'] = data['grade'].map({'A':1,'B':2,'C':3,'D':4,'E':5,'F':6,'G':7})

`pd.get_dummies`: https://blog.csdn.net/maymay_/article/details/80198468

In [ ]:
# 类型数在2之上，又不是高维稀疏的,且纯分类特征
for data in [data_train, data_test_a]:
    data = pd.get_dummies(data, columns=['subGrade', 'homeOwnership', 'verificationStatus', 'purpose', 'regionCode'], drop_first=True)

## Define a function here for finding outliers.

Especially for numerical data that are outside of the range (3 * standard error). 

In [ ]:
def find_outliers_by_3segama(data,fea):
    stdError = np.std(data[fea])
    meanVal = np.mean(data[fea])
    lowerBound = meanVal - 3*stdError
    higherBound = meanVal + 3*stdError
    data[fea+'_outliers'] = data[fea].apply(lambda x: str('异常值') if x < lowerBound or x > higherBound else str('正常值'))
    return data

## See some of the labels' situation. 

`isDefault` is the label.

The meaning of the outputs of the following cell is that: _in each column, rows are grouped by `正常值` or `异常值`, and the `sum()` will be the sum of multiple '1's and '0's. E.g., in column `interestRate_outliers`, there are 150000+ 正常值 rows that are positive labeled, and there are 2916 异常值 rows that are positive labeled._

In [ ]:
data_train = data_train.copy()
for fea in numerical_fea:
    data_train = find_outliers_by_3segama(data_train,fea)
    print(data_train[fea+'_outliers'].value_counts())
    print("-"*10)
    print(data_train.groupby(fea+'_outliers')['isDefault'].sum())
    print('*'*10)
    print()

正常值    800000
Name: id_outliers, dtype: int64
----------
id_outliers
正常值    159610
Name: isDefault, dtype: int64
**********

正常值    800000
Name: loanAmnt_outliers, dtype: int64
----------
loanAmnt_outliers
正常值    159610
Name: isDefault, dtype: int64
**********

正常值    800000
Name: term_outliers, dtype: int64
----------
term_outliers
正常值    159610
Name: isDefault, dtype: int64
**********

正常值    794259
异常值      5741
Name: interestRate_outliers, dtype: int64
----------
interestRate_outliers
异常值      2916
正常值    156694
Name: isDefault, dtype: int64
**********

正常值    792046
异常值      7954
Name: installment_outliers, dtype: int64
----------
installment_outliers
异常值      2152
正常值    157458
Name: isDefault, dtype: int64
**********

正常值    800000
Name: employmentTitle_outliers, dtype: int64
----------
employmentTitle_outliers
正常值    159610
Name: isDefault, dtype: int64
**********

正常值    799701
异常值       299
Name: homeOwnership_outliers, dtype: int64
----------
homeOwnership_outliers
异常值      

In [ ]:
data_train.head(3).append(data_train.tail(3))

,id,loanAmnt,term,interestRate,installment,grade,subGrade,employmentTitle,employmentLength,homeOwnership,annualIncome,verificationStatus,issueDate,isDefault,purpose,postCode,regionCode,dti,delinquency_2years,ficoRangeLow,ficoRangeHigh,openAcc,pubRec,pubRecBankruptcies,revolBal,revolUtil,totalAcc,initialListStatus,applicationType,earliesCreditLine,title,policyCode,n0,n1,n2,n3,n4,n5,n6,n7,...,loanAmnt_outliers,term_outliers,interestRate_outliers,installment_outliers,employmentTitle_outliers,homeOwnership_outliers,annualIncome_outliers,verificationStatus_outliers,purpose_outliers,postCode_outliers,regionCode_outliers,dti_outliers,delinquency_2years_outliers,ficoRangeLow_outliers,ficoRangeHigh_outliers,openAcc_outliers,pubRec_outliers,pubRecBankruptcies_outliers,revolBal_outliers,revolUtil_outliers,totalAcc_outliers,initialListStatus_outliers,applicationType_outliers,title_outliers,policyCode_outliers,n0_outliers,n1_outliers,n2_outliers,n3_outliers,n4_outliers,n5_outliers,n6_outliers,n7_outliers,n8_outliers,n9_outliers,n10_outliers,n11_outliers,n12_outliers,n13_outliers,n14_outliers
0,0,35000.0,5,19.52,917.97,5,E2,320.0,2.0,2,110000.0,2,2014-07-01,1,1,137.0,32,17.05,0.0,730.0,734.0,7.0,0.0,0.0,24178.0,48.9,27.0,0,0,2001,1.0,1.0,0.0,2.0,2.0,2.0,4.0,9.0,8.0,4.0,...,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值
1,1,18000.0,5,18.49,461.90,4,D2,219843.0,5.0,0,46000.0,2,2012-08-01,0,0,156.0,18,27.83,0.0,700.0,704.0,13.0,0.0,0.0,15096.0,38.9,18.0,1,0,2002,1723.0,1.0,0.0,3.0,5.0,5.0,10.0,7.0,7.0,7.0,...,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值
2,2,12000.0,5,16.99,298.17,4,D3,31698.0,8.0,0,74000.0,2,2015-10-01,0,0,337.0,14,22.77,0.0,675.0,679.0,11.0,0.0,0.0,4606.0,51.8,27.0,0,0,2006,0.0,1.0,0.0,0.0,3.0,3.0,0.0,0.0,21.0,4.0,...,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值
799997,799997,6000.0,3,13.33,203.12,3,C3,2582.0,10.0,1,65000.0,2,2015-10-01,1,0,47.0,17,12.11,1.0,670.0,674.0,5.0,0.0,0.0,6381.0,51.9,36.0,1,0,2002,0.0,1.0,2.0,1.0,4.0,4.0,1.0,4.0,26.0,4.0,...,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值
799998,799998,19200.0,3,6.92,592.14,1,A4,151.0,10.0,0,96000.0,2,2015-02-01,0,4,34.0,18,29.25,0.0,675.0,679.0,16.0,0.0,0.0,69702.0,61.3,37.0,1,0,1994,4.0,1.0,0.0,5.0,8.0,8.0,7.0,10.0,6.0,12.0,...,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值
799999,799999,9000.0,3,11.06,294.91,2,B3,13.0,5.0,0,120000.0,0,2018-08-01,0,4,62.0,13,8.99,0.0,695.0,699.0,7.0,0.0,0.0,8420.0,72.6,13.0,0,0,2002,4.0,1.0,2.0,2.0,3.0,3.0,2.0,3.0,4.0,4.0,...,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值,正常值


## Delete 异常值, which is abnormal values. 

Only the rows that all numerical columns are normal values will be kept. Other rows will be given up. 

**Mind this**: sometimes abnormal values cannot be removed. They should also be kept, because sometimes the abnormal values can lead to discoveries. 

In [ ]:
#删除异常值
for fea in numerical_fea:
    data_train = data_train[data_train[fea+'_outliers']=='正常值']
    data_train = data_train.reset_index(drop=True) 

In [ ]:
data_train.shape

(612742, 89)

## Distribute the data into bins 

Following cells are fake codes. 

**Mind this**: Multiple methods can be used. 

In [ ]:
# 通过除法映射到间隔均匀的分箱中，每个分箱的取值范围都是loanAmnt/1000
data['loanAmnt_bin1'] = np.floor_divide(data['loanAmnt'], 1000)

In [ ]:
## 通过对数函数映射到指数宽度分箱
data['loanAmnt_bin2'] = np.floor(np.log10(data['loanAmnt']))

The `pd.qcut()` is cut the numbers into bins. 

The functionality of parameter `labels=False/True` can be seen from the following cells.  

In [ ]:
data['loanAmnt_bin3'] = pd.qcut(data['loanAmnt'], 10, labels=False)
data["loanAmnt_bin3"].head()

0    5
1    7
2    4
3    6
4    9
Name: loanAmnt_bin3, dtype: int64

In [ ]:
data_train_tst = data_train.copy()
data_train_tst['loanAmnt_bin3'] = pd.qcut(data_train_tst['loanAmnt'], 10, labels=False)
data_train_tst["loanAmnt_bin3"].head()

0    9
1    7
2    4
3    0
4    4
Name: loanAmnt_bin3, dtype: int64

In [ ]:
data_train_tst = data_train.copy()
data_train_tst['loanAmnt_bin3'] = pd.qcut(data_train_tst['loanAmnt'], 10)
data_train_tst["loanAmnt_bin3"].head()

0    (25000.0, 40000.0]
1    (17500.0, 20000.0]
2    (10000.0, 12000.0]
3     (499.999, 5000.0]
4    (10000.0, 12000.0]
Name: loanAmnt_bin3, dtype: category
Categories (10, interval[float64]): [(499.999, 5000.0] < (5000.0, 6500.0] < (6500.0, 8500.0] <
                                     (8500.0, 10000.0] ... (15000.0, 17500.0] < (17500.0, 20000.0] <
                                     (20000.0, 25000.0] < (25000.0, 40000.0]]

## Combinatorial features

（交互特征）is the combination of original features. https://www.msra.cn/zh-cn/news/features/kdd-2018-xdeepfm#:~:text=%E7%89%B9%E5%BE%81%E4%BA%A4%E4%BA%92%E6%8C%87%E7%9A%84%E6%98%AF,user_id%2C%20item_id%5D%E7%9A%84%E8%81%94%E7%B3%BB%E3%80%82

**Mind this**: there may be multiple ways of combining features. Try more methods. 

In [ ]:
## 这里的target mean, 就跟我之前那个kaggle项目用的target encoding在思路上有一定的类似之处.
## 但是这里的具体算法又不太一样. 
## 这里的算法是: 
### 将col进行分组, 求每一个分组里面的label的总和的平均值是多少
## 然后再映射一下, 构建新的列. 
## 这里就是一种combination的方法了, 可以学习一下. 
for col in ['grade', 'subGrade']: 
    temp_dict = data_train.groupby([col])['isDefault'].agg(['mean']).reset_index().rename(columns={'mean': col + '_target_mean'})
    temp_dict.index = temp_dict[col].values
    print(temp_dict.head())
    temp_dict = temp_dict[col + '_target_mean'].to_dict()
    print(col, " ", temp_dict, sum(temp_dict.values()), "\n")

    data_train[col + '_target_mean'] = data_train[col].map(temp_dict)
    data_test_a[col + '_target_mean'] = data_test_a[col].map(temp_dict)


   grade  grade_target_mean
1      1           0.059838
2      2           0.131210
3      3           0.224522
4      4           0.304227
5      5           0.386234
grade   {1: 0.05983754010496838, 2: 0.1312100457861034, 3: 0.22452249131030422, 4: 0.30422665003427624, 5: 0.3862340548898338, 6: 0.4509921313718782, 7: 0.45314009661835747} 2.010163010115722 

   subGrade  subGrade_target_mean
A1       A1              0.032617
A2       A2              0.044493
A3       A3              0.053966
A4       A4              0.065532
A5       A5              0.083522
subGrade   {'A1': 0.03261699574560925, 'A2': 0.04449302765303711, 'A3': 0.05396644867785044, 'A4': 0.06553160215580597, 'A5': 0.08352181466548661, 'B1': 0.10053276344700961, 'B2': 0.1094614852398524, 'B3': 0.12811052672894263, 'B4': 0.14836514352965255, 'B5': 0.16260956738554866, 'C1': 0.1912715651761416, 'C2': 0.2040048707887972, 'C3': 0.22468573405604889, 'C4': 0.2515842653973075, 'C5': 0.2622187489824493, 'D1': 0.27944432385500

In [ ]:
# 其他衍生变量 mean 和 std
## 我认为, 这一部的操作增加的是什么呢? 
## 除号右边那个部分得到的是: 按照nx分组, 每一组求一个平均数, 然后把这个平均数赋给每一行数据, 让每一行数据自己认领一个属于自己的平均数值. 
## 结合上除号, 就得到的是: 每一行数据的等级分除以每一行数据对应的平均分, 得到的商. 
## 学习一下, 如果要进行特征组合, 多采用这里面用到的方法, 诸如groupby, transform之类. 
for df in [data_train, data_test_a]:
    for item in ['n0','n1','n2','n3','n4','n5','n6','n7','n8','n9','n10','n11','n12','n13','n14']:
        df['grade_to_mean_' + item] = df['grade'] / df.groupby([item])['grade'].transform('mean')
        df['grade_to_std_' + item] = df['grade'] / df.groupby([item])['grade'].transform('std')

In [ ]:
data_train[['grade', 'subGrade', 'grade_target_mean', "subGrade_target_mean"]].head()

,grade,subGrade,grade_target_mean,subGrade_target_mean
0,5,E2,0.386234,0.380444
1,4,D2,0.304227,0.298190
2,4,D3,0.304227,0.302541
3,1,A4,0.059838,0.065532
4,3,C3,0.224522,0.224686


In [ ]:
# df.groupby(["n0"])['grade'].transform('mean')

In [ ]:
# df.groupby(["n0"])['grade'].mean()

## Change the strings into numbers. 

http://sofasofa.io/forum_main_post.php?postid=1001659

In [ ]:
#label-encode:subGrade,postCode,title
# 高维类别特征需要进行转换
for col in tqdm(['employmentTitle', 'postCode', 'title','subGrade']):
    le = LabelEncoder()
    le.fit(list(data_train[col].astype(str).values) + list(data_test_a[col].astype(str).values))
    data_train[col] = le.transform(list(data_train[col].astype(str).values))
    data_test_a[col] = le.transform(list(data_test_a[col].astype(str).values))
print('Label Encoding 完成')

100%|██████████| 4/4 [00:05<00:00,  1.49s/it]

Label Encoding 完成


## ~Fill the empty cells again~

Use the row above the n/a row to fill the n/a cells. In another word, fill the n/a with previous value.

There are some cells used previously. I think after they are used, there is no need to run the empty cell again. 

**Mind this**: other methods may used. 

In [ ]:
# data_train = data_train.fillna(axis = 0, method = "ffill")

## Delete the features that will not be used to fit the model

No `xx_outliers` features, no labels, no original dates. 

In [ ]:
features = [f for f in data_train.columns if f not in ['id','issueDate','isDefault'] and '_outliers' not in f]
x_train = data_train[features]
x_test = data_test_a[features]
y_train = data_train['isDefault']

In [ ]:
# for data in [data_train, data_test_a]:
#     data.drop(["issueDate", "id"], axis = 1, inplace = True)

## Optimize the memory size of the dataset 

In [ ]:
# reduce_mem_usage 函数通过调整数据类型，帮助我们减少数据在内存中占用的空间
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() 
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() 
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
x_train_small = reduce_mem_usage(x_train)
x_test_small = reduce_mem_usage(x_test)

Memory usage of dataframe is 377449200.00 MB
Memory usage after optimization is: 92524170.00 MB
Decreased by 75.5%
Memory usage of dataframe is 123200128.00 MB
Memory usage after optimization is: 35000128.00 MB
Decreased by 71.6%


## ~Calculating covariance~

The covariance is more useful when you want to select features. I guess. Perhaps we don't have to anything here. 

In [ ]:
# x_train = data_train.drop(['isDefault'], axis=1)
# #计算协方差
# data_corr = x_train.corrwith(data_train["isDefault"]) #计算相关性
# data_corr

In [ ]:
# result = pd.DataFrame(columns=['features', 'corr'])
# result['features'] = data_corr.index
# result['corr'] = data_corr.values
# result

Visualize the correlation: 

In [ ]:
# numerical_fea

In [ ]:
# # 当然也可以直接看图
# numerical_fea.remove("id")
# data_numeric = data_train[numerical_fea]
# correlation = data_numeric.corr()

# f, ax = plt.subplots(figsize = (7, 7))
# plt.title('Correlation of Numeric Features with Price',y=1,size=16)
# sns.heatmap(correlation,square = True,  vmax=0.8)

In [ ]:
# list(data_train.columns)

## ~Select some features~

**Mind this**: other methods may be used. But I have tried the following code, it seems to degrade the performance. So I don't suggest to use the feature selection if we have enough computational resources. 

In [ ]:
# from sklearn.feature_selection import SelectKBest
# from scipy.stats import pearsonr
# #选择K个最好的特征，返回选择特征后的数据
# #第一个参数为计算评估特征是否好的函数，该函数输入特征矩阵和目标向量，
# #输出二元组（评分，P值）的数组，数组第i项为第i个特征的评分和P值。在此定义为计算相关系数
# #参数k为选择的特征个数

# selector = SelectKBest(k=50)
# selector.fit(
#     x_train_small,
#     y_train
# )

In [ ]:
# colNums = selector.get_support(True)

In [ ]:
# selectedFeatures = []
# for i, col in enumerate(list(x_train_small.columns)):
#     if i in colNums:
#         selectedFeatures.append(col)
# len(selectedFeatures)

In [ ]:
# x_train_small_featureSelected = x_train_small[selectedFeatures]
# x_test_small_featureSelected = x_train_small[selectedFeatures]

In [ ]:
## Some other feature selection methods

# from sklearn.feature_selection import VarianceThreshold
# #其中参数threshold为方差的阈值
# VarianceThreshold(threshold=3).fit_transform(
#     data_train[['grade', 'subGrade', 'grade_target_mean', "subGrade_target_mean"]],
#     data_train["isDefault"]
# )

# Save preprocessed data

In [ ]:
x_train.head()

,loanAmnt,term,interestRate,installment,grade,subGrade,employmentTitle,employmentLength,homeOwnership,annualIncome,verificationStatus,purpose,postCode,regionCode,dti,delinquency_2years,ficoRangeLow,ficoRangeHigh,openAcc,pubRec,pubRecBankruptcies,revolBal,revolUtil,totalAcc,initialListStatus,applicationType,earliesCreditLine,title,policyCode,n0,n1,n2,n3,n4,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14,issueDateDT,grade_target_mean,subGrade_target_mean,grade_to_mean_n0,grade_to_std_n0,grade_to_mean_n1,grade_to_std_n1,grade_to_mean_n2,grade_to_std_n2,grade_to_mean_n3,grade_to_std_n3,grade_to_mean_n4,grade_to_std_n4,grade_to_mean_n5,grade_to_std_n5,grade_to_mean_n6,grade_to_std_n6,grade_to_mean_n7,grade_to_std_n7,grade_to_mean_n8,grade_to_std_n8,grade_to_mean_n9,grade_to_std_n9,grade_to_mean_n10,grade_to_std_n10,grade_to_mean_n11,grade_to_std_n11,grade_to_mean_n12,grade_to_std_n12,grade_to_mean_n13,grade_to_std_n13,grade_to_mean_n14,grade_to_std_n14
0,35008.0,5,19.515625,918.0000,5,21,161280,2.0,2,110000.0,2,1,43,32,17.046875,0.0,730.0,734.0,7.0,0.0,0.0,24178.0,48.90625,27.0,0,0,2001,1,1.0,0.0,2.0,2.0,2.0,4.0,9.0,8.0,4.0,12.0,2.0,7.0,0.0,0.0,0.0,2.0,2587,0.386230,0.380371,1.875977,3.992188,1.875000,4.054688,1.942383,4.023438,1.942383,4.023438,1.869141,3.947266,1.897461,4.054688,1.866211,4.019531,1.840820,4.074219,1.851562,4.039062,1.938477,4.023438,1.841797,4.109375,1.852539,4.011719,1.852539,4.011719,1.857422,4.003906,1.856445,3.992188
1,18000.0,5,18.484375,462.0000,4,16,89538,5.0,0,46000.0,2,0,64,18,27.828125,0.0,700.0,704.0,13.0,0.0,0.0,15096.0,38.90625,18.0,1,0,2002,5768,1.0,0.0,3.0,5.0,5.0,10.0,7.0,7.0,7.0,13.0,5.0,13.0,0.0,0.0,0.0,2.0,1888,0.304199,0.298096,1.500977,3.193359,1.502930,3.185547,1.503906,3.173828,1.503906,3.173828,1.567383,3.205078,1.511719,3.138672,1.515625,3.099609,1.500977,3.140625,1.517578,3.085938,1.503906,3.173828,1.484375,3.173828,1.482422,3.207031,1.482422,3.207031,1.486328,3.205078,1.485352,3.193359
2,12000.0,5,16.984375,298.2500,4,17,159367,8.0,0,74000.0,2,0,265,14,22.765625,0.0,675.0,679.0,11.0,0.0,0.0,4606.0,51.81250,27.0,0,0,2006,0,1.0,0.0,0.0,3.0,3.0,0.0,0.0,21.0,4.0,5.0,3.0,11.0,0.0,0.0,0.0,4.0,3044,0.304199,0.302490,1.500977,3.193359,1.360352,2.998047,1.533203,3.242188,1.533203,3.242188,1.273438,3.070312,1.162109,3.175781,1.480469,3.125000,1.472656,3.259766,1.406250,3.253906,1.531250,3.244141,1.503906,3.089844,1.482422,3.207031,1.482422,3.207031,1.486328,3.205078,1.315430,3.146484
3,2050.0,3,7.691406,63.9375,1,3,59830,9.0,0,35000.0,0,0,465,14,17.484375,0.0,755.0,759.0,12.0,0.0,0.0,3111.0,8.50000,23.0,0,0,2006,0,1.0,0.0,1.0,3.0,3.0,7.0,11.0,3.0,10.0,18.0,3.0,12.0,0.0,0.0,0.0,3.0,2679,0.059845,0.065552,0.375244,0.798340,0.368164,0.796387,0.383301,0.810547,0.383301,0.810547,0.380615,0.806641,0.385010,0.802734,0.368408,0.819336,0.369873,0.798340,0.377930,0.799316,0.382812,0.811035,0.370117,0.799316,0.370605,0.801758,0.370605,0.801758,0.371582,0.801270,0.344238,0.793457
4,11504.0,3,14.976562,398.5000,3,12,85242,1.0,1,30000.0,2,0,3,4,32.593750,0.0,665.0,669.0,8.0,1.0,1.0,14021.0,59.68750,33.0,1,0,1994,0,1.0,0.0,4.0,4.0,4.0,4.0,16.0,10.0,5.0,21.0,4.0,8.0,0.0,0.0,0.0,2.0,2406,0.224487,0.224731,1.125977,2.394531,1.113281,2.431641,1.133789,2.439453,1.133789,2.439453,1.121094,2.369141,1.198242,2.400391,1.121094,2.388672,1.106445,2.451172,1.144531,2.402344,1.133789,2.441406,1.104492,2.447266,1.111328,2.406250,1.111328,2.406250,1.114258,2.402344,1.114258,2.394531


In [ ]:
x_train.shape

(612742, 77)

In [ ]:
x_train_small.to_csv("preprocessedData/x_train_small.csv", index=False)
x_test_small.to_csv("preprocessedData/x_test_small.csv", index=False)
y_train.to_csv("preprocessedData/y_train.csv", index=False)